# 서울시 미용실 데이터 - NAVER Map(v4) 데이터 수집
## 작성자(최종수정일) : 서동원(2020.04.17)
- 서울 열린데이터 광장 API를 이용해 수집한 데이터(이하 공공데이터) 활용
- 공공데이터를 네이버 지역검색 API에 적용시켜 네이버에 등록된 미용실 데이터 수집

In [304]:
# !pip install selenium

In [305]:
# !pip install bs4

In [306]:
# !pip install pyperclip

In [307]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import requests
import time
import datetime
import math
import glob
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기
pd.set_option('display.max_rows',100)

from selenium import webdriver
from bs4 import BeautifulSoup
import tqdm
import datetime

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyperclip

from selenium.common import exceptions

## (1) 공공데이터 가져와 전처리 하기

- 이하 전처리 작업에서 매번 공공 API를 호출할 필요 없다.
- 공공 API로 호출한 데이터를 저장한 csv파일을 읽어와 진핼할 것
- 일단 2020년 4월 9일자 기준 데이터로 전처리를 진행하겠다.

In [308]:
file_list = glob.glob('./public*')
file_list

['.\\public_api_column_info.csv',
 '.\\public_api_error_info.csv',
 '.\\public_api_key_info.csv',
 '.\\public_api_seoul_salon_data_2020-03-26.csv',
 '.\\public_api_seoul_salon_data_2020-03-30.csv',
 '.\\public_api_seoul_salon_data_2020-04-01.csv',
 '.\\public_api_seoul_salon_data_2020-04-03.csv',
 '.\\public_api_seoul_salon_data_2020-04-09.csv']

In [309]:
file_path = file_list[-1]
file_path

'.\\public_api_seoul_salon_data_2020-04-09.csv'

In [310]:
# csv 파일 읽어서 DF에 대입
public_salon_raw = pd.read_csv(file_path, index_col=0)

In [311]:
# 데이터 확인
public_salon_raw.head(2)

,지역,CGG_CODE,SNT_COB_CODE,YY,UPSO_SNO,SNT_COB_NM,UPSO_GSL_YMD,UPSO_NM,TRDP_AREA,UPSO_SITE_TELNO,BMAN_STDT,BUP_NM,SITE_STDT,ADMDNG_NM,DCB_YMD,DCB_WHY,SNT_UPTAE_NM,ED_FIN_YMD,GAEKSIL,HANSHIL,YANGSIL,CHAIR_NUM,YOKSIL,BALHANSIL_YN,WASHMC_NUM,PERM_NT_NO,KOR_FRGNR_GBN,NTN,SITE_ADDR_RD,SITE_ADDR
0,강남구,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,19.80,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
1,강남구,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,65.67,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩


In [312]:
# 컬럼을 보기 쉽게 변환하기
# 컬럼명 정보가 담긴 csv 파일 읽어오기 
col_convert = pd.read_csv('./public_api_column_info.csv')
col_convert

,출력명,출력설명
0,CGG_CODE,시군구코드
1,SNT_COB_CODE,업종코드
2,YY,년도
3,UPSO_SNO,업소일련번호
4,SNT_COB_NM,업종명
5,UPSO_GSL_YMD,신고일자
6,UPSO_NM,업소명
7,TRDP_AREA,면적
8,UPSO_SITE_TELNO,소재지전화번호
9,BMAN_STDT,영업자시작일


In [313]:
# 두 컬럼의 값을 dict 객체로 변환
col_dict = dict(zip(col_convert.출력명, col_convert.출력설명))
col_dict

{'CGG_CODE': '시군구코드',
 'SNT_COB_CODE': '업종코드',
 'YY': '년도',
 'UPSO_SNO': '업소일련번호',
 'SNT_COB_NM': '업종명',
 'UPSO_GSL_YMD': '신고일자',
 'UPSO_NM': '업소명',
 'TRDP_AREA': '면적',
 'UPSO_SITE_TELNO': '소재지전화번호',
 'BMAN_STDT': '영업자시작일',
 'BUP_NM': '법인명',
 'SITE_STDT': '소재지시작일',
 'ADMDNG_NM': '행정동명',
 'DCB_YMD': '폐업일자',
 'DCB_WHY': '폐업사유',
 'SNT_UPTAE_NM': '업태명',
 'ED_FIN_YMD': '위생교육수료일자',
 'GAEKSIL': '객실수',
 'HANSHIL': '한실수',
 'YANGSIL': '양실수',
 'CHAIR_NUM': '의자수',
 'YOKSIL': '욕실수',
 'BALHANSIL_YN': '발한실',
 'WASHMC_NUM': '세탁기수',
 'PERM_NT_NO': '허가(신고)번호',
 'KOR_FRGNR_GBN': '내외국인구분',
 'NTN': '국적',
 'SITE_ADDR_RD': '소재지도로명',
 'SITE_ADDR': '소재지지번'}

In [314]:
# 컬럼명 변경하기
public_salon_raw.rename(columns=col_dict, inplace=True)
public_salon_raw.head(1)

,지역,시군구코드,업종코드,년도,업소일련번호,업종명,신고일자,업소명,면적,소재지전화번호,영업자시작일,법인명,소재지시작일,행정동명,폐업일자,폐업사유,업태명,위생교육수료일자,객실수,한실수,양실수,의자수,욕실수,발한실,세탁기수,허가(신고)번호,내외국인구분,국적,소재지도로명,소재지지번
0,강남구,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,19.8,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩


In [315]:
public_salon_raw.tail(1)

,지역,시군구코드,업종코드,년도,업소일련번호,업종명,신고일자,업소명,면적,소재지전화번호,영업자시작일,법인명,소재지시작일,행정동명,폐업일자,폐업사유,업태명,위생교육수료일자,객실수,한실수,양실수,의자수,욕실수,발한실,세탁기수,허가(신고)번호,내외국인구분,국적,소재지도로명,소재지지번
28286,중랑구,3060000,211,2020,10,미용업(일반),20200310,우주헤어,19.65,NaN,20200310,NaN,20200310,상봉제1동,NaN,NaN,일반미용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3060000-211-2020-00010,내국인,NaN,"서울특별시 중랑구 망우로43길 3, 1층 좌측호 (상봉동)",서울특별시 중랑구 상봉동 116번지 45호


In [316]:
public_salon_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28287 entries, 0 to 28286
Data columns (total 30 columns):
지역          28287 non-null object
시군구코드       28287 non-null int64
업종코드        28287 non-null int64
년도          28287 non-null int64
업소일련번호      28287 non-null int64
업종명         28287 non-null object
신고일자        28287 non-null int64
업소명         28287 non-null object
면적          28287 non-null float64
소재지전화번호     17853 non-null object
영업자시작일      28287 non-null int64
법인명         1 non-null object
소재지시작일      28287 non-null int64
행정동명        28208 non-null object
폐업일자        8619 non-null float64
폐업사유        8611 non-null object
업태명         28287 non-null object
위생교육수료일자    11101 non-null float64
객실수         28287 non-null float64
한실수         28287 non-null float64
양실수         28287 non-null float64
의자수         28287 non-null float64
욕실수         28287 non-null float64
발한실         27961 non-null object
세탁기수        28287 non-null float64
허가(신고)번호    28287 non-null object
내외국인구분     

In [317]:
# 총 데이터 개수 확인
pre_data_count = len(public_salon_raw)
print('총 데이터 수 :', len(public_salon_raw))

총 데이터 수 : 28287


---
- 업소명에 '?'문자가 있는 부분을 조사해보니 '샾'을 인식하지 못해 물음표로 처리된 것으로 판단.
- '?' 문자를 '샵'으로 변환

In [318]:
# 업소명에 '?' 문자를 '샵' 문자로 치환하기
public_salon_raw['업소명'] = public_salon_raw['업소명'].apply(lambda x: re.sub('[？]|[?]', '샵', x))


# '?'문자가 '샵'으로 전부 변환되지 않는 문제를 발견. 조치 필요 !!!!
# 일반 문자'?' 가 아닌 특수문자'？' 이어서 대체가 안된 것

---
- 중복값 처리

In [319]:
# 모든 컬럼값이 중복인 row 제거
public_salon_raw.drop_duplicates(inplace=True)

# 중복 제거 후 데이터 개수 확인
print('중복여부로 제거 전 데이터 수 :', pre_data_count)
print('중복여부로 제거 후 데이터 수 :', len(public_salon_raw))
print('중복여부로 제거된 데이터 수 :', pre_data_count - len(public_salon_raw))
pre_data_count = len(public_salon_raw)

중복여부로 제거 전 데이터 수 : 28287
중복여부로 제거 후 데이터 수 : 28231
중복여부로 제거된 데이터 수 : 56


---
- 폐업점 데이터 처리

In [320]:
# '폐업일자'의 값이 NaN이면 영업중
# '폐업일자'의 값이 NaN이 아니면 폐업된 것으로 판단

# '폐업일자'에 값이 존재하는 폐업된 업장이 얼마나 있는지 확인 
public_salon_raw[public_salon_raw['폐업일자'].notnull()]['폐업일자']

5        20200113.0
7        20130829.0
8        20150821.0
21       20180222.0
25       20190415.0
            ...    
28206    20190827.0
28227    20191119.0
28258    20190802.0
28265    20200213.0
28268    20200206.0
Name: 폐업일자, Length: 8616, dtype: float64

In [321]:
# 폐업한 미용실 정보가 8591개나 된다.

In [322]:
# '폐업일자' 컬럼이 NaN인, 영업중인 행만 불리언 색인 -> public_salon_raw에 대입
public_salon_raw = public_salon_raw[public_salon_raw['폐업일자'].isnull()]

# 데이터 개수 확인

# 폐업여부로 삭제 후 데이터 개수 확인
print('폐업여부로 제거 전 데이터 수 :', pre_data_count)
print('폐업여부로 제거된 데이터 수 :', pre_data_count - len(public_salon_raw))
print('폐업여부로 제거 후 데이터 수 :', len(public_salon_raw))
pre_data_count = len(public_salon_raw)

폐업여부로 제거 전 데이터 수 : 28231
폐업여부로 제거된 데이터 수 : 8616
폐업여부로 제거 후 데이터 수 : 19615


In [323]:
# 업태명에 어떤 값들이 있는지 확인
public_salon_raw['업태명'].unique()

array(['일반미용업', '네일아트업', '기타', '메이크업업', '일반이용업', '숙박업 기타'], dtype=object)

In [324]:
# '업태명'이 '일반미용업', '일반이용업' 인 데이터만 남기기
public_salon_raw = public_salon_raw[public_salon_raw['업태명'].isin(['일반미용업', '일반이용업'])]

# 업태명으로 삭제 후 데이터 개수 확인
print('업태명으로 제거 전 데이터 수 :', pre_data_count)
print('업태명으로 제거 후 데이터 수 :', len(public_salon_raw))
print('업태명으로 제거된 데이터 수 :', pre_data_count - len(public_salon_raw))
pre_data_count = len(public_salon_raw)

업태명으로 제거 전 데이터 수 : 19615
업태명으로 제거 후 데이터 수 : 19355
업태명으로 제거된 데이터 수 : 260


In [325]:
# 컬럼명 확인하기
public_salon_raw.columns

Index(['지역', '시군구코드', '업종코드', '년도', '업소일련번호', '업종명', '신고일자', '업소명', '면적',
       '소재지전화번호', '영업자시작일', '법인명', '소재지시작일', '행정동명', '폐업일자', '폐업사유', '업태명',
       '위생교육수료일자', '객실수', '한실수', '양실수', '의자수', '욕실수', '발한실', '세탁기수',
       '허가(신고)번호', '내외국인구분', '국적', '소재지도로명', '소재지지번'],
      dtype='object')

In [326]:
public_salon_raw[public_salon_raw['업태명'] == '일반이용업']

,지역,시군구코드,업종코드,년도,업소일련번호,업종명,신고일자,업소명,면적,소재지전화번호,영업자시작일,법인명,소재지시작일,행정동명,폐업일자,폐업사유,업태명,위생교육수료일자,객실수,한실수,양실수,의자수,욕실수,발한실,세탁기수,허가(신고)번호,내외국인구분,국적,소재지도로명,소재지지번
5474,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,97.85,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5475,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,24.00,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5476,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,97.85,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5477,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,24.00,0262057133,20170515,NaN,20180903,화곡제2동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 강서로8길 68, 1층 (화곡동)",서울특별시 강서구 화곡동 882번지 26호 (지상 1층)
5478,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,97.85,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
5479,강서구,3150000,211,2017,26,미용업(일반),20170515,승민헤어,24.00,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
5480,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,97.85,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
5481,강서구,3150000,211,2017,26,미용업(일반),20170515,규림헤어겔러리,24.00,0262057133,20170515,NaN,20170515,화곡제8동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,4.0,0.0,N,0.0,3150000-211-2017-00026,내국인,NaN,"서울특별시 강서구 곰달래로25길 77, 2층 (화곡동)",서울특별시 강서구 화곡동 340번지 59호 2층
6204,관악구,3200000,211,2008,1,미용업(일반),20080708,신보배 헤어,28.05,02 883 9489,20171026,NaN,20080708,은천동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,5.0,0.0,NaN,0.0,3200000-211-2008-00001,내국인,NaN,"서울특별시 관악구 은천로 95, 벽산블루밍상가 501동 308호 (봉천동)",서울특별시 관악구 봉천동 1718번지 벽산블루밍상가
6205,관악구,3200000,211,2008,1,미용업(일반),20080708,머리못하는집,28.05,02 883 9489,20171026,NaN,20080708,은천동,NaN,NaN,일반이용업,NaN,0.0,0.0,0.0,5.0,0.0,NaN,0.0,3200000-211-2008-00001,내국인,NaN,"서울특별시 관악구 은천로 95, 벽산블루밍상가 501동 308호 (봉천동)",서울특별시 관악구 봉천동 1718번지 벽산블루밍상가


In [327]:
# 필요한 컬럼들로만 재색인
public_salon = public_salon_raw.reindex(columns=['지역'
                                                 ,'업소명'
                                                 ,'소재지전화번호'
                                                 ,'행정동명'
                                                 ,'소재지도로명'
                                                 ,'소재지지번'
                                                ])
public_salon.head(3)

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번
0,강남구,백민재헤어샵,02 5451290,청담동,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
1,강남구,백민재헤어샵,02 5451290,청담동,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
2,강남구,백민재헤어샵,02 5451290,삼성1동,"서울특별시 강남구 삼성로 649, (삼성동,(상아APT 상가 107호))",서울특별시 강남구 삼성동 19번지 4호 (상아APT 상가 107호)


In [328]:
# DF의 head()로 보기에는 문제가 없지만
# 직접 찍어보니 공백이 여러개 중복되어 있는 경우가 있다.
public_salon['소재지지번'][0]

'서울특별시 강남구 청담동  133번지 3호  화천회관빌딩'

## 공백 제거
- 각 컬럼에 들어있는 값(문자열)의 
    - (1) 중복 공백을 1개로 줄인 뒤
    - (2) 앞뒤 공백 제거

In [329]:
# '업소명' 
public_salon['업소명'] = public_salon['업소명'].apply(lambda x: re.sub('\s+', ' ', x).strip() if type(x)=='str' else x)

# '소재지전화번호' 
public_salon['소재지전화번호'] = public_salon['소재지전화번호'].apply(lambda x: re.sub('\s+', ' ', x).strip() if type(x)=='str' else x)
# public_salon['소재지전화번호'] = public_salon['소재지전화번호'].apply(lambda x: remove_double_whitespace(x) if x else x)

# '행정동명' 
public_salon['행정동명'] = public_salon['행정동명'].apply(lambda x: re.sub('\s+', ' ', x).strip() if type(x)=='str' else x)
# public_salon['행정동명'] = public_salon['행정동명'].apply(lambda x: remove_double_whitespace(x) if x else x)

# '소재지도로명' 
public_salon['소재지도로명'] = public_salon['소재지도로명'].apply(lambda x: re.sub('\s+', ' ', x).strip() if type(x)=='str' else x)

# '소재지지번' 
public_salon['소재지지번'] = public_salon['소재지지번'].apply(lambda x: re.sub('\s+', ' ', x).strip() if type(x)=='str' else x)
# public_salon['소재지지번'] = public_salon['소재지지번'].apply(lambda x: remove_double_whitespace(x) if x else x)

In [330]:
# 작업 확인
public_salon['소재지지번'][0]

'서울특별시 강남구 청담동  133번지 3호  화천회관빌딩'

In [331]:
pre_data = len(public_salon)

## 중복 제거
- raw 데이터에서 필요한 컬럼만 재색인 했는데 다시 한번 중복 제거를 하겠다.

In [332]:
# 중복 제거
public_salon.drop_duplicates(inplace=True)

print('중복 제거 전 데이터 수 :', pre_data)
print('중복 제거된 데이터 수 :', pre_data - len(public_salon))
print('중복 제거 후 데이터 수 :', len(public_salon))

중복 제거 전 데이터 수 : 19355
중복 제거된 데이터 수 : 3441
중복 제거 후 데이터 수 : 15914


In [333]:
public_salon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15914 entries, 0 to 28286
Data columns (total 6 columns):
지역         15914 non-null object
업소명        15914 non-null object
소재지전화번호    10287 non-null object
행정동명       15873 non-null object
소재지도로명     15860 non-null object
소재지지번      15873 non-null object
dtypes: object(6)
memory usage: 870.3+ KB


In [334]:
# index 빠진 이빨 채우기
# index를 0부터 데이터 길이만큼 재설정
public_salon.index=range(len(public_salon))

In [335]:
# 데이터 개수 확인
len(public_salon)

15914

In [336]:
# 데이터(인덱스가 잘 변경 됐는지) 확인
# 데이터길이와 마지막 인덱스 비교 
public_salon.tail(3)

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번
15911,중랑구,에이데이즈 헤어살롱,NaN,신내1동,"서울특별시 중랑구 봉화산로 232, 2층 (신내동)",서울특별시 중랑구 신내동 405번지 5호
15912,중랑구,지노헤어 상봉점,NaN,상봉제2동,"서울특별시 중랑구 망우로 282-1, (상봉동)",서울특별시 중랑구 상봉동 115번지 14호
15913,중랑구,우주헤어,NaN,상봉제1동,"서울특별시 중랑구 망우로43길 3, 1층 좌측호 (상봉동)",서울특별시 중랑구 상봉동 116번지 45호


In [337]:
# '소재지도로명'에 누락값 있는지 확인
no_addrRoad = public_salon[public_salon['소재지도로명'].isnull()]
print("'소재지도로명'이 누락된 데이터 수 :", len(no_addrRoad))

'소재지도로명'이 누락된 데이터 수 : 54


In [338]:
# '소재지지번'에 누락값 있는지 확인
no_addrOrigin = public_salon[public_salon['소재지지번'].isnull()]
print("'소재지지번'이 누락된 데이터 수 :", len(no_addrOrigin))

'소재지지번'이 누락된 데이터 수 : 41


In [339]:
# '소재지도로명' 없는 데이터 중 '소재지지번'이 없는 데이터
print("'소재지도로명'이 누락된 데이터 중 '소재지지번'도 동시 누락된 수 :", len(no_addrRoad[no_addrRoad['소재지지번'].isnull()]))

'소재지도로명'이 누락된 데이터 중 '소재지지번'도 동시 누락된 수 : 0


- 특이사항
- 소재지도로명'과 '소재지지번'의 누락값이 일부 발견되었으나
- 두 컬럼의 값이 모두 누락된 경우는 없음
- 특이사항으로 '소재지지번'이 누락된 경우 '행정동명' 컬럼 또한 같이 누락됨

In [340]:
# 결측치 존재하는지 확인
(public_salon.isnull()).sum()

지역            0
업소명           0
소재지전화번호    5627
행정동명         41
소재지도로명       54
소재지지번        41
dtype: int64

In [341]:
# 데이터 정보 확인
public_salon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15914 entries, 0 to 15913
Data columns (total 6 columns):
지역         15914 non-null object
업소명        15914 non-null object
소재지전화번호    10287 non-null object
행정동명       15873 non-null object
소재지도로명     15860 non-null object
소재지지번      15873 non-null object
dtypes: object(6)
memory usage: 746.1+ KB


In [342]:
# 크롤링에 사용할 DF
# 저장할 파일 이름 생성
base_name = './naver_map_seoul_salon_input_data'
collect_day = file_path.split('_')[-1].split('.')[0]
work_day = datetime.datetime.now().strftime('%Y-%m-%d')

file_name = '_'.join([base_name, collect_day, work_day]) + '.csv'
file_name

'./naver_map_seoul_salon_input_data_2020-04-09_2020-04-20.csv'

In [343]:
# 크롤링에 사용할 DF 별도 저장
public_salon.to_csv(
      file_name
    , sep=','
    , encoding='utf-8'
)

In [344]:
# 네이버 검색 결과를 별도 저장할 DF 생성
# (첫번째 실행때만 DF 생성 코드 실행할 것)
naver_search_all = DataFrame(columns=['업소ID'
                                    , '업소명'
                                    , '소재지전화번호'
                                    , '소재지도로명' 
                                    , '카테고리'
                                    , '검색주소타입'
                                    , '검색인덱스'
                                    , '검색어'
                                    , '검색건수'
                                    ])

naver_search_all

# # 두번째 실행부터는 저장된 csv파일 불러오기
# # 저장된 네이버 검색 결과 csv 파일 불러와 DF 생성
# naver_search_all = pd.read_csv('./naver_map_seoul_salon_data_2020-04-18.csv', encoding='utf-8').drop(columns='Unnamed: 0')
# naver_search_all

,업소ID,업소명,소재지전화번호,소재지도로명,카테고리,검색주소타입,검색인덱스,검색어,검색건수


In [345]:
# 첫번째 실행때만 DF 생성 코드 실행할 것
# 검색결과가 300건이 넘는 경우 api 사용 중 에러 발생.
# 이를 검색하지않고 따로 저장하는 DF 생성
naver_search_rejected = DataFrame(columns=['지역'
                                        , '업소명'
                                        , '검색주소'
                                        , '검색주소타입'
                                        , '검색어'
                                        , '검색건수' 
                                        , '원인'
                                         ])
naver_search_rejected

# # 두번째 실행부터는 저장된 csv 파일 불러오기
# # 검색결과 300건이 넘어 크롤링을 보류한 데이터 csv 파일 불러와 DF 생성
# naver_search_rejected = pd.read_csv('./naver_map_seoul_salon_rejected_data_2020-04-18.csv', encoding='utf-8').drop(columns='Unnamed: 0')
# naver_search_rejected

,지역,업소명,검색주소,검색주소타입,검색어,검색건수,원인


---
# (2) 네이버 지도(v4)에서 Selenium, BeautifulSoup 이용 
- 데이터 크롤링

In [346]:
def send_to_input_box(keyword, input_box, fix_display_search, driver, search_button_xpath):
    # 입력박스에 검색어 입력
    print('검색어 :', keyword)
    input_box.clear()
    input_box.send_keys(keyword)
    
    
    # '현 지도에서 장소검색' 체크가 되어있지 않다면 체크
    if not fix_display_search.is_selected():
        fix_display_search.click()
    
    time.sleep(np.random.rand())
    
    # 검색 버튼 클릭
    driver.find_element_by_xpath(search_button_xpath).click()
    driver.implicitly_wait(5)
    
    # 현재 페이지 소스 가져와 parsing하기
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [347]:
def crawl_naver_map_v4(series, collect_day, driver):

    print('try row-num :', series.name)
    
    global naver_search_all  # 검색결과 저장 DF 전역변수 사용
    global naver_search_rejected  # 검색거부 저장 DF 전역변수 사용
    
    global count_success
    global count_over_fifty
    global count_search_zero
    global count_avoid_capcha
    
    # 입력박스 찾아서 주소 입력
    input_box = driver.find_element_by_id('search-input')
    input_box.clear()
    search_addr=''
    search_addr_type=0
    if type(series['소재지도로명']) == str:
        search_addr = series['소재지도로명']
        search_addr_type = 1
    else:
        search_addr = series['소재지지번']
        search_addr_type = 2
    
    input_box.send_keys(search_addr)
    
    # '현 지도에서 장소검색' 체크되어있다면 체크 해제
    # '(현 지도에서 장소검색' 체크박스 id : searchCurr)
    fix_display_search = driver.find_element_by_id('searchCurr')
    if fix_display_search.is_selected():
        fix_display_search.click()
    
    time.sleep(np.random.rand())
    
    # 검색 버튼 클릭
    search_button_xpath = """//*[@id="header"]/div[1]/fieldset/button"""
    driver.find_element_by_xpath(search_button_xpath).click()
    driver.implicitly_wait(5)
    
    # 입력박스에 검색어 입력
    keyword = '%s' % series['업소명'] + ' 미용실'
    soup = send_to_input_box(keyword, input_box, fix_display_search, driver, search_button_xpath)
    
    # 검색결과 건수 가져오기
    # 총 페이지수 계산하기
    if soup.find('span', 'n'):
        # 검색결과 건수
        total_result = int(soup.find('span', 'n').find('em').text.replace(',',''))
        if total_result <= 50:
            print('검색결과 %s건' % total_result)
            # 페이지 수
            total_page = math.ceil(total_result / 10)
        else:
            print('검색결과 %s건' % total_result)
            print('[%s] 검색결과 50건 초과 !!' % datetime.datetime.now())
            
            keyword = '"%s"' % series['업소명'] + ' 미용실'
            
            print('재시도 -', end=' ')
            soup = send_to_input_box(keyword, input_box, fix_display_search, driver, search_button_xpath)
           
            if soup.find('span', 'n'):
                # 검색결과 건수
                total_result = int(soup.find('span', 'n').find('em').text.replace(',',''))
                if total_result <= 50:
                    print('검색결과 %s건' % total_result)
                    # 페이지 수
                    total_page = math.ceil(total_result / 10)
                else:
                    print('검색결과 %s건' % total_result)
                    print('[%s] 검색결과 50건 초과 !!' % datetime.datetime.now())
            
                    naver_search_rejected = naver_search_rejected.append(DataFrame({
                          '지역' : series['지역']
                        , '업소명' : series['업소명']
                        , '검색주소' : search_addr
                        , '검색주소타입' : search_addr_type
                        , '검색어' : keyword
                        , '검색건수' : total_result
                        , '원인' : '검색건수 50건 초과'
                        }
                        , index=[series.name]
                        
                    ), sort=False
                    )     
                    # 오늘 날짜 출력
                    today = datetime.datetime.now().strftime('%Y-%m-%d')

                    # 검색결과가 없어 검색 실패한 데이터 csv 파일로 저장
                    naver_search_rejected.to_csv(
                          './naver_map_seoul_salon_outout_rejected_data_%s.csv' % (collect_day, today)
                        , sep=','
                        , encoding='utf-8'
                    )
                    
                    count_over_fifty+=1
                    print(f'성공 : {count_success} | 0건 검색 : {count_search_zero} | 50건이상 검색 : {count_over_fifty} | 캡챠 우회 : {count_avoid_capcha}\n')
                    
                    return
      
    else:
        print('검색결과 0건')
        print('[%s] 검색결과 없음 !!' % datetime.datetime.now())    

        naver_search_rejected = naver_search_rejected.append(DataFrame({
              '지역' : series['지역']
            , '업소명' : series['업소명']
            , '검색주소' : search_addr
            , '검색주소타입' : search_addr_type
            , '검색어' : keyword
            , '검색건수' : 0
            , '원인' : '검색결과 없음'
            }
            , index=[series.name]

        ), sort=False
        )     

        # 오늘 날짜 출력
        today = datetime.datetime.now().strftime('%Y-%m-%d')

        # 검색결과가 없어 검색 실패한 데이터 csv 파일로 저장
        naver_search_rejected.to_csv(
              './naver_map_seoul_salon_output_rejected_data_%s_%s.csv' % (collect_day, today)
            , sep=','
            , encoding='utf-8'
        )

        count_search_zero+=1
        print(f'성공 : {count_success} | 0건 검색 : {count_search_zero} | 50건이상 검색 : {count_over_fifty} | 캡챠 우회 : {count_avoid_capcha}\n')

        return
    
    # 결과 정보 담겨있는 html 찾기
    shop_list = soup.find_all('dl', 'lsnx_det')
    
    title_list=[]
    addrRoad_list=[]
    tel_list=[]
    category_list=[]
    
    # 다음버튼 xpath : //*[@id="panel"]/div[2]/div[1]/div[2]/div[2]/div/div/a[1]
    next_page_xpath = '//*[@id="panel"]/div[2]/div[1]/div[2]/div[2]/div/div/a'
    
    fail_count=0
    id_list=[]
    for count_page in range(1, total_page+1):
        
        while 1:
            try:
                time.sleep(np.random.rand()+1)
                html = driver.page_source
                time.sleep(0.3)
                soup = BeautifulSoup(html, 'html.parser')

                info_block = soup.find('ul', 'lst_site')
                info_list = info_block.find_all('li')
                info_list = list(filter(lambda x: x.get_attribute_list('data-id')[0], info_list))
                id_list = id_list + list(map(lambda x: x.get_attribute_list('data-id')[0], info_list))
                shop_list = soup.find_all('dl', 'lsnx_det')
                break

            except AttributeError:
                fail_count+=1
                if fail_count == 3:
                    print('연결 3회 실패')
                    raise EOFError
                print('page_source 불러오기 실패. 재시도')
                continue
        
        for shop in shop_list:
            
            # 업소명 추가
            title_list.append(shop.find('a').text.strip())
            
            # 주소(도로명) 추가
            tmp_addr = shop.find('dd', 'addr').text.strip()
            addrRoad_list.append(re.sub('\s{3,}.+', '', tmp_addr))
            
            # 전화번호 추가
            try:
                tel_list.append(shop.find('dd', 'tel').text.strip())
            except:
                tel_list.append(np.nan)
            
            # 카테고리 추가
            category_list.append(shop.find('dd', 'cate').text)

        if count_page == total_page:
            break
        else:
            xpath_num = count_page%5 if count_page%5 != 0 else 5
            driver.find_element_by_xpath(next_page_xpath + '[%s]' % (xpath_num)).click()
            time.sleep(0.5)

            
    # 네이버 검색결과 DataFrame에 추가(저장)
    naver_search_all = naver_search_all.append(pd.DataFrame({
              '업소ID' : id_list
            , '업소명' : title_list
            , '소재지전화번호' : tel_list
            , '소재지도로명' : addrRoad_list
            , '카테고리' : category_list
            , '검색주소타입' : search_addr_type
            , '검색인덱스' : series.name
            , '검색어' : keyword
            , '검색건수' : total_result
            }
        )
        , ignore_index=True
        , sort=False
    )

    # 오늘 날짜 출력
    today = datetime.datetime.now().strftime('%Y-%m-%d')

    # 현재까지의 네이버 검색 결과를 csv 파일로 저장
    naver_search_all.to_csv(
          './naver_map_seoul_salon_output_data_%s_%s.csv' % (collect_day, today)
        , sep=','
        , encoding='utf-8'
    )
    
    count_success+=1
    print('[%s] 검색 및 저장 성공 !!' % datetime.datetime.now())
    print(f'성공 : {count_success} | 0건 검색 : {count_search_zero} | 50건이상 검색 : {count_over_fifty} | 캡챠 우회 : {count_avoid_capcha}\n')
    
    return

In [348]:
def clipboard_input(user_xpath, user_input):

        pyperclip.copy(user_input)
        driver.find_element_by_xpath(user_xpath).click()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

        time.sleep(np.random.rand()+1)

In [349]:
def run_naver_map_search(dataframe, collect_day, start, end, restart=0):
    
    if restart !=0:
        start = restart
    
    options = webdriver.ChromeOptions()

    # options.add_arguement('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')
    options.add_argument("Accept=text/html,application/xhtml+xml,application/xml;q=0.9,imgwebp,*/*;q=0.8")
    driver = webdriver.Chrome('c:/chromedriver', options=options)
    driver.get('https://v4.map.naver.com/')   
    driver.add_cookie(driver.get_cookies()[0])

    # '오늘 하루 그만 보기' 체크박스 xpath : //*[@id="dday_popup"]/div[2]/div[2]/label/span
    elem_checkbox = driver.find_element_by_xpath('//*[@id="dday_popup"]/div[2]/div[2]/label/span')

    # 체크박스가 체크되어있지 않으면 클릭
    if not elem_checkbox.is_selected():
        elem_checkbox.click()

    # x버튼 xpath : //*[@id="dday_popup"]/div[2]/button/span[1]
    # 창닫기 x버튼 클릭
    driver.find_element_by_xpath('//*[@id="dday_popup"]/div[2]/button/span[1]').click()
    
    global count_success
    global count_over_fifty
    global count_search_zero
    global count_avoid_capcha
    
    count_success = 0
    count_over_fifty = 0
    count_search_zero = 0
    count_avoid_capcha = 0
    
    login = {
          "id" : "qlrthdthd"
        , "pw" : "adminuser"
    }


    for i in tqdm.tnrange(len(dataframe[start:end])):
        try:
            crawl_naver_map_v4(dataframe[start:end].iloc[i], collect_day, driver)

        except (exceptions.StaleElementReferenceException, exceptions.NoSuchElementException):
            clipboard_input('//*[@id="id"]', login.get("id"))
            clipboard_input('//*[@id="pw"]', login.get("pw"))
            driver.find_element_by_xpath('//*[@id="log.login"]').click()
            driver.implicitly_wait(3)
            time.sleep(0.5)
            
            count_avoid_capcha+=1
            print('캡차 우회 !!')
            print(f'성공 : {count_success} | 0건 검색 : {count_search_zero} | 50건이상 검색 : {count_over_fifty} | 캡챠 우회 : {count_avoid_capcha}\n')
            
            crawl_naver_map_v4(dataframe[start:end].iloc[i], collect_day, driver)
            continue

    driver.close()
    print('\n검색 종료!')

In [350]:
start = 0
end = 20
restart = 0

run_naver_map_search(public_salon, collect_day, start, end, restart)

try row-num : 0
검색어 : 백민재헤어샵 미용실
검색결과 0건
[2020-04-20 02:01:31.486828] 검색결과 없음 !!
성공 : 0 | 0건 검색 : 1 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 1
검색어 : 백민재헤어샵 미용실
검색결과 0건
[2020-04-20 02:01:35.140717] 검색결과 없음 !!
성공 : 0 | 0건 검색 : 2 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 2
검색어 : 들어가도싱글나가도벙글 미용실 미용실
검색결과 0건
[2020-04-20 02:01:38.054511] 검색결과 없음 !!
성공 : 0 | 0건 검색 : 3 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 3
검색어 : 조희미용실 미용실
검색결과 1건
[2020-04-20 02:01:45.855979] 검색 및 저장 성공 !!
성공 : 1 | 0건 검색 : 3 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 4
검색어 : 정정원헤어룩 미용실
검색결과 1건
[2020-04-20 02:01:51.402472] 검색 및 저장 성공 !!
성공 : 2 | 0건 검색 : 3 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 5
검색어 : 박승철헤어스투디오 청담점 미용실
검색결과 1건
[2020-04-20 02:01:56.380804] 검색 및 저장 성공 !!
성공 : 3 | 0건 검색 : 3 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 6
검색어 : 박승철헤어스튜디오 미용실
검색결과 1건
[2020-04-20 02:02:00.770002] 검색 및 저장 성공 !!
성공 : 4 | 0건 검색 : 3 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 7
검색어 : 이가자헤어비스 미용실
검색결과 1건
[2020-04-20 02:02:04.763909] 검색 

In [62]:
# driver.close()

In [ ]:
# try row-num : 30
# 검색어 : 누벨바그헤어살롱
# ---------------------------------------------------------------------------
# NoSuchElementException                    Traceback (most recent call last)
# <ipython-input-158-0aa4ca422ad7> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon)):
# ---> 26     crawl_naver_map_v4(public_salon.iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-157-32c1e58d3a88> in crawl_naver_map_v4(series)
#      12 
#      13     # 입력박스 찾아서 주소 입력
# ---> 14     input_box = driver.find_element_by_id('search-input')
#      15     input_box.clear()
#      16     input_box.send_keys(series['소재지도로명'])

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in find_element_by_id(self, id_)
#     358             element = driver.find_element_by_id('foo')
#     359         """
# --> 360         return self.find_element(by=By.ID, value=id_)
#     361 
#     362     def find_elements_by_id(self, id_):

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in find_element(self, by, value)
#     976         return self.execute(Command.FIND_ELEMENT, {
#     977             'using': by,
# --> 978             'value': value})['value']
#     979 
#     980     def find_elements(self, by=By.ID, value=None):

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in execute(self, driver_command, params)
#     319         response = self.command_executor.execute(driver_command, params)
#     320         if response:
# --> 321             self.error_handler.check_response(response)
#     322             response['value'] = self._unwrap_value(
#     323                 response.get('value', None))

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py in check_response(self, response)
#     240                 alert_text = value['alert'].get('text')
#     241             raise exception_class(message, screen, stacktrace, alert_text)
# --> 242         raise exception_class(message, screen, stacktrace)
#     243 
#     244     def _value_or_default(self, obj, key, default):

# NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search-input"]"}
#   (Session info: chrome=81.0.4044.92)

#======================================================================
#======================================================================

#try row-num : 255
#검색어 : 뮤엘
#검색결과 1건
#---------------------------------------------------------------------------
#AttributeError                            Traceback (most recent call last)
#<ipython-input-161-719cad261a20> in <module>
#     24 tmp_list = []
#     25 for i in tqdm.tnrange(len(public_salon[30:])):
#---> 26     crawl_naver_map_v4(public_salon[30:].iloc[i])
#     27 
#     28 driver.close()
#
#<ipython-input-160-8ffe9b918ef8> in crawl_naver_map_v4(series)
#    105 
#    106         info_block = soup.find('ul', 'lst_site')
#--> 107         info_list = info_block.find_all('li')
#    108         info_list = list(filter(lambda x: x.get_attribute_list('data-id')[0], info_list))
#    109         id_list = id_list + list(map(lambda x: x.get_attribute_list('data-id')[0], info_list))
#
#AttributeError: 'NoneType' object has no attribute 'find_all'

#==========================================================================
#==========================================================================

# start : 255
# try row-num : 260
# 검색어 : 헤어스토리
# ---------------------------------------------------------------------------
# KeyboardInterrupt                         Traceback (most recent call last)
# <ipython-input-103-2e560687da1a> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[255:])):
# ---> 26     crawl_naver_map_v4(public_salon[255:].iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-102-8ffe9b918ef8> in crawl_naver_map_v4(series)
#      22         fix_display_search.click()
#      23 
# ---> 24     time.sleep(np.random.rand(1)[0]+np.random.rand(1)[0]+2)
#      25 
#      26     # 검색 버튼 클릭

# KeyboardInterrupt:


# try row-num : 294
# 검색어 : 또 헤어
# 검색결과 1건
# ---------------------------------------------------------------------------
# KeyboardInterrupt                         Traceback (most recent call last)
# <ipython-input-36-b752e1f8e17b> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[260:])):
# ---> 26     crawl_naver_map_v4(public_salon[260:].iloc[i])
#      27 
#      28 driver.close()
# 
# <ipython-input-35-8ffe9b918ef8> in crawl_naver_map_v4(series)
#     100 
#     101         driver.implicitly_wait(5)
# --> 102         time.sleep(np.random.rand(1)[0]+1)
#     103         html = driver.page_source
#     104         soup = BeautifulSoup(html, 'html.parser')
# 
# KeyboardInterrupt: 

# ==================================================================================
# ==================================================================================

# start : 294
# try row-num : 333
# 검색어 : 누벨바그 헤어살롱(역삼점)
# 검색결과 1건
# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# <ipython-input-38-2e8dad201efa> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[294:])):
# ---> 26     crawl_naver_map_v4(public_salon[294:].iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-37-248d09942161> in crawl_naver_map_v4(series)
#     105 
#     106         info_block = soup.find('ul', 'lst_site')
# --> 107         info_list = info_block.find_all('li')
#     108         info_list = list(filter(lambda x: x.get_attribute_list('data-id')[0], info_list))
#     109         id_list = id_list + list(map(lambda x: x.get_attribute_list('data-id')[0], info_list))

# AttributeError: 'NoneType' object has no attribute 'find_all'


#=====================================================================
#=====================================================================

# start : 333
# try row-num : 381
# 검색어 : 프리미엄헤어살롱(Primeum hair salon)미용실 늘솜
# 검색결과 1건
# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# <ipython-input-40-ebcc212188d5> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[333:])):
# ---> 26     crawl_naver_map_v4(public_salon[333:].iloc[i])
#      27 
#      28 driver.close()
# 
# <ipython-input-37-248d09942161> in crawl_naver_map_v4(series)
#     105 
#     106         info_block = soup.find('ul', 'lst_site')
# --> 107         info_list = info_block.find_all('li')
#     108         info_list = list(filter(lambda x: x.get_attribute_list('data-id')[0], info_list))
#     109         id_list = id_list + list(map(lambda x: x.get_attribute_list('data-id')[0], info_list)) 
# 
# AttributeError: 'NoneType' object has no attribute 'find_all'



#============================================================================================================
#============================================================================================================

# start : 381
# try row-num : 385
# 검색어 : 한양스파미용실
# ---------------------------------------------------------------------------
# KeyboardInterrupt                         Traceback (most recent call last)
# <ipython-input-50-0237943e4872> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[381:])):
# ---> 26     crawl_naver_map_v4(public_salon[381:].iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-49-9ade12b41aad> in crawl_naver_map_v4(series)
#      22         fix_display_search.click()
#      23 
# ---> 24     time.sleep(np.random.rand(1)[0]+np.random.rand(1)[0]+1)
#      25 
#      26     # 검색 버튼 클릭

# KeyboardInterrupt: 

#==================================================================================
#==================================================================================

# start : 385
# try row-num : 481
# 검색어 : 비더블유와이 뷰티 컴퍼니(BWY Beauty Company)
# 검색결과 1건
# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# <ipython-input-52-e8060564b029> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[385:])):
# ---> 26     crawl_naver_map_v4(public_salon[385:].iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-51-dc4151e52a63> in crawl_naver_map_v4(series)
#     108 
#     109         info_block = soup.find('ul', 'lst_site')
# --> 110         info_list = info_block.find_all('li')
#     111         info_list = list(filter(lambda x: x.get_attribute_list('data-id')[0], info_list))
#     112         id_list = id_list + list(map(lambda x: x.get_attribute_list('data-id')[0], info_list))

# AttributeError: 'NoneType' object has no attribute 'find_all'


#==================================================================================
#==================================================================================

# 여기서부터 검색건수 가져오기 전에 time.sleep(0.5)를 주어 AttributeError가 나지 않는다.
# 여기 이전 데이터의 검색건수는 틀릴 수 있음

# start : 481
# try row-num : 2240
# 검색어 : 킴스헤어살롱
# ---------------------------------------------------------------------------
# KeyboardInterrupt                         Traceback (most recent call last)
# <ipython-input-57-313def9677b8> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[481:])):
# ---> 26     crawl_naver_map_v4(public_salon[481:].iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-56-fbfe27fb10c0> in crawl_naver_map_v4(series)
#      22         fix_display_search.click()
#      23 
# ---> 24     time.sleep(np.random.rand(1)[0]+np.random.rand(1)[0]+1)
#      25 
#      26     # 검색 버튼 클릭

# KeyboardInterrupt:

#==================================================================================
#==================================================================================

# start : 2240
#try row-num : 4402
#검색어 : 해어화
#---------------------------------------------------------------------------
#TypeError                                 Traceback (most recent call last)
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py in _make_request(self, conn, method, url, timeout, chunked, **httplib_request_kw)
#    376             try:  # Python 2.7, use buffering of HTTP responses
#--> 377                 httplib_response = conn.getresponse(buffering=True)
#    378             except TypeError:  # Python 3
#
#TypeError: getresponse() got an unexpected keyword argument 'buffering'
#
#During handling of the above exception, another exception occurred:
#
#KeyboardInterrupt                         Traceback (most recent call last)
#<ipython-input-41-d6dae26ad23a> in <module>
#     24 tmp_list = []
#     25 for i in tqdm.tnrange(len(public_salon[2240:])):
#---> 26     crawl_naver_map_v4(public_salon[2240:].iloc[i])
#     27 
#     28 driver.close()
#
#<ipython-input-38-02482cc7d112> in crawl_naver_map_v4(series)
#     20     fix_display_search = driver.find_element_by_id('searchCurr')
#     21     if fix_display_search.is_selected():
#---> 22         fix_display_search.click()
#     23 
#     24     time.sleep(np.random.rand())
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py in click(self)
#     78     def click(self):
#     79         """Clicks the element."""
#---> 80         self._execute(Command.CLICK_ELEMENT)
#     81 
#     82     def submit(self):
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py in _execute(self, command, params)
#    631             params = {}
#    632         params['id'] = self._id
#--> 633         return self._parent.execute(command, params)
#    634 
#    635     def find_element(self, by=By.ID, value=None):
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in execute(self, driver_command, params)
#    317 
#    318         params = self._wrap_value(params)
#--> 319         response = self.command_executor.execute(driver_command, params)
#    320         if response:
#    321             self.error_handler.check_response(response)
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\remote_connection.py in execute(self, command, params)
#    372         data = utils.dump_json(params)
#    373         url = '%s%s' % (self._url, path)
#--> 374         return self._request(command_info[0], url, body=data)
#    375 
#    376     def _request(self, method, url, body=None):
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\remote_connection.py in _request(self, method, url, body)
#    395 
#    396         if self.keep_alive:
#--> 397             resp = self._conn.request(method, url, body=body, headers=headers)
#    398 
#    399             statuscode = resp.status
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\request.py in request(self, method, url, fields, headers, **urlopen_kw)
#     70             return self.request_encode_body(method, url, fields=fields,
#     71                                             headers=headers,
#---> 72                                             **urlopen_kw)
#     73 
#     74     def request_encode_url(self, method, url, fields=None, headers=None,
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\request.py in request_encode_body(self, method, url, fields, headers, encode_multipart, multipart_boundary, **urlopen_kw)
#    148         extra_kw.update(urlopen_kw)
#    149 
#--> 150         return self.urlopen(method, url, **extra_kw)
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\poolmanager.py in urlopen(self, method, url, redirect, **kw)
#    322             response = conn.urlopen(method, url, **kw)
#    323         else:
#--> 324             response = conn.urlopen(method, u.request_uri, **kw)
#    325 
#    326         redirect_location = redirect and response.get_redirect_location()
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
#    598                                                   timeout=timeout_obj,
#    599                                                   body=body, headers=headers,
#--> 600                                                   chunked=chunked)
#    601 
#    602             # If we're going to release the connection in ``finally:``, then
#
#~\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py in _make_request(self, conn, method, url, timeout, chunked, **httplib_request_kw)
#    378             except TypeError:  # Python 3
#    379                 try:
#--> 380                     httplib_response = conn.getresponse()
#    381                 except Exception as e:
#    382                     # Remove the TypeError from the exception chain in Python 3;
#
#~\AppData\Local\Continuum\anaconda3\lib\http\client.py in getresponse(self)
#   1334         try:
#   1335             try:
#-> 1336                 response.begin()
#   1337             except ConnectionError:
#   1338                 self.close()
#
#~\AppData\Local\Continuum\anaconda3\lib\http\client.py in begin(self)
#    304         # read until we get a non-100 response
#    305         while True:
#--> 306             version, status, reason = self._read_status()
#    307             if status != CONTINUE:
#    308                 break
#
#~\AppData\Local\Continuum\anaconda3\lib\http\client.py in _read_status(self)
#    265 
#    266     def _read_status(self):
#--> 267         line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
#    268         if len(line) > _MAXLINE:
#    269             raise LineTooLong("status line")
#
#~\AppData\Local\Continuum\anaconda3\lib\socket.py in readinto(self, b)
#    587         while True:
#    588             try:
#--> 589                 return self._sock.recv_into(b)
#    590             except timeout:
#    591                 self._timeout_occurred = True
#
#KeyboardInterrupt:

#==================================================================================
#==================================================================================

# start : 4402
# try row-num : 5250
# 검색어 : The끌림헤어
# 검색결과 1건
# page_source 불러오기 실패. 재시도
# ---------------------------------------------------------------------------
# KeyboardInterrupt                         Traceback (most recent call last)
# <ipython-input-74-b70f755b06fa> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[4402:])):
# ---> 26     crawl_naver_map_v4(public_salon[4402:].iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-73-02482cc7d112> in crawl_naver_map_v4(series)
#     103                 time.sleep(np.random.rand()+1)
#     104                 html = driver.page_source
# --> 105                 time.sleep(0.3)
#     106                 soup = BeautifulSoup(html, 'html.parser')
#     107 

# KeyboardInterrupt: 

#==================================================================================
#==================================================================================

# start : 5250
# try row-num : 6158
# 검색어 : 헤어셰프
# ---------------------------------------------------------------------------
# StaleElementReferenceException            Traceback (most recent call last)
# <ipython-input-76-86f5992ab743> in <module>
#      24 tmp_list = []
#      25 for i in tqdm.tnrange(len(public_salon[5250:])):
# ---> 26     crawl_naver_map_v4(public_salon[5250:].iloc[i])
#      27 
#      28 driver.close()

# <ipython-input-75-faddb60f975e> in crawl_naver_map_v4(series)
#      30 
#      31     # 입력박스에 검색어 입력
# ---> 32     input_box.clear()
#      33     input_box.send_keys(series['업소명'])
#      34 

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py in clear(self)
#      93     def clear(self):
#      94         """Clears the text if it's a text entry element."""
# ---> 95         self._execute(Command.CLEAR_ELEMENT)
#      96 
#      97     def get_property(self, name):

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py in _execute(self, command, params)
#     631             params = {}
#     632         params['id'] = self._id
# --> 633         return self._parent.execute(command, params)
#     634 
#     635     def find_element(self, by=By.ID, value=None):

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in execute(self, driver_command, params)
#     319         response = self.command_executor.execute(driver_command, params)
#     320         if response:
# --> 321             self.error_handler.check_response(response)
#     322             response['value'] = self._unwrap_value(
#     323                 response.get('value', None))

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py in check_response(self, response)
#     240                 alert_text = value['alert'].get('text')
#     241             raise exception_class(message, screen, stacktrace, alert_text)
# --> 242         raise exception_class(message, screen, stacktrace)
#     243 
#     244     def _value_or_default(self, obj, key, default):

# StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
#   (Session info: chrome=81.0.4044.113)


#==================================================================================
#==================================================================================

# start : 6158
# try row-num : 6158
# 검색어 : 헤어셰프
# [0] 캡차 우회!

# try row-num : 6159  -> 왜 다시 우히한거 재시도 안하지?
    
# try row-num : 6585
# 검색어 : 장안
# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-64-9063abaa98c6> in <module>
#      33 for i in tqdm.tnrange(len(public_salon[6158:])):
#      34     try:
# ---> 35         crawl_naver_map_v4(public_salon[6158:].iloc[i])
#      36 
#      37     except exceptions.StaleElementReferenceException:

# <ipython-input-42-d3ec62e36dd9> in crawl_naver_map_v4(series)
#      52     if soup.find('span', 'n'):
#      53         # 검색결과 건수
# ---> 54         total_result = int(soup.find('span', 'n').find('em').text)
#      55         print('검색결과 %s건' % total_result)
#      56         # 페이지 수

# ValueError: invalid literal for int() with base 10: '1,454'

#==================================================================================
#==================================================================================

# start : 6585
# try row-num : 7545
# ---------------------------------------------------------------------------
# NoSuchElementException                    Traceback (most recent call last)
# <ipython-input-80-24ddee5e39c8> in <module>
#      33 for i in tqdm.tnrange(len(public_salon[6585:])):
#      34     try:
# ---> 35         crawl_naver_map_v4(public_salon[6585:].iloc[i])
#      36 
#      37     except exceptions.StaleElementReferenceException:

# <ipython-input-79-1b7f961ac223> in crawl_naver_map_v4(series)
#      12 
#      13     # 입력박스 찾아서 주소 입력
# ---> 14     input_box = driver.find_element_by_id('search-input')
#      15     input_box.clear()
#      16     input_box.send_keys(series['소재지도로명'])

# ~\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in find_element_by_id(self, id_)
#     358             element = driver.find_element_by_id('foo')
#     359         """
# --> 360         return self.find_element(by=By.ID, value=id_)
#     361 
#     362     def find_elements_by_id(self, id_):

# ~\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in find_element(self, by, value)
#     976         return self.execute(Command.FIND_ELEMENT, {
#     977             'using': by,
# --> 978             'value': value})['value']
#     979 
#     980     def find_elements(self, by=By.ID, value=None):

# ~\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in execute(self, driver_command, params)
#     319         response = self.command_executor.execute(driver_command, params)
#     320         if response:
# --> 321             self.error_handler.check_response(response)
#     322             response['value'] = self._unwrap_value(
#     323                 response.get('value', None))

# ~\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py in check_response(self, response)
#     240                 alert_text = value['alert'].get('text')
#     241             raise exception_class(message, screen, stacktrace, alert_text)
# --> 242         raise exception_class(message, screen, stacktrace)
#     243 
#     244     def _value_or_default(self, obj, key, default):

# NoSuchElementException: Message: no such element: Unable to locate element: {"method":"id","selector":"search-input"}
#   (Session info: chrome=80.0.3987.149)
#   (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 10.0.17763 x86_64)

#==================================================================================
#==================================================================================

# start : 7545
# try row-num : 9661
# 검색어 : 선헤어랜드
# ---------------------------------------------------------------------------
# KeyboardInterrupt                         Traceback (most recent call last)
# <ipython-input-83-66ad3c43bd48> in <module>
#      33 for i in tqdm.tnrange(len(public_salon[7545:])):
#      34     try:
# ---> 35         crawl_naver_map_v4(public_salon[7545:].iloc[i])
#      36 
#      37     except (exceptions.StaleElementReferenceException, exceptions.NoSuchElementException):

# <ipython-input-79-1b7f961ac223> in crawl_naver_map_v4(series)
#      53     time.sleep(1)
#      54     html = driver.page_source
# ---> 55     soup = BeautifulSoup(html, 'html.parser')
#      56 
#      57     # 검색결과 건수 가져오기

# ~\anaconda3\lib\site-packages\bs4\__init__.py in __init__(self, markup, features, builder, parse_only, from_encoding, exclude_encodings, element_classes, **kwargs)
#     323             self.reset()
#     324             try:
# --> 325                 self._feed()
#     326                 success = True
#     327                 break

# ~\anaconda3\lib\site-packages\bs4\__init__.py in _feed(self)
#     397         self.builder.reset()
#     398 
# --> 399         self.builder.feed(self.markup)
#     400         # Close out any unfinished strings and close all the open tags.
#     401         self.endData()

# ~\anaconda3\lib\site-packages\bs4\builder\_htmlparser.py in feed(self, markup)
#     337         parser.soup = self.soup
#     338         try:
# --> 339             parser.feed(markup)
#     340             parser.close()
#     341         except HTMLParseError as e:

# ~\anaconda3\lib\html\parser.py in feed(self, data)
#     109         """
#     110         self.rawdata = self.rawdata + data
# --> 111         self.goahead(0)
#     112 
#     113     def close(self):

# ~\anaconda3\lib\html\parser.py in goahead(self, end)
#     169             if startswith('<', i):
#     170                 if starttagopen.match(rawdata, i): # < + letter
# --> 171                     k = self.parse_starttag(i)
#     172                 elif startswith("</", i):
#     173                     k = self.parse_endtag(i)

# ~\anaconda3\lib\html\parser.py in parse_starttag(self, i)
#     343             self.handle_startendtag(tag, attrs)
#     344         else:
# --> 345             self.handle_starttag(tag, attrs)
#     346             if tag in self.CDATA_CONTENT_ELEMENTS:
#     347                 self.set_cdata_mode(tag)

# ~\anaconda3\lib\site-packages\bs4\builder\_htmlparser.py in handle_starttag(self, name, attrs, handle_empty_element)
#     123             sourcepos=sourcepos
#     124         )
# --> 125         if tag and tag.is_empty_element and handle_empty_element:
#     126             # Unlike other parsers, html.parser doesn't send separate end tag
#     127             # events for empty-element tags. (It's handled in

# ~\anaconda3\lib\site-packages\bs4\element.py in __bool__(self)
#    1332         return x in self.contents
#    1333 
# -> 1334     def __bool__(self):
#    1335         "A tag is non-None even if it has no contents."
#    1336         return True

# KeyboardInterrupt:

#==================================================================================
#==================================================================================


# try row-num : 11642
# 검색어 : 헤어 갤리(Hair Gally)
# [1981] 캡차 우회!


# start : 9661
# try row-num : 12709
# ---------------------------------------------------------------------------
# ElementClickInterceptedException          Traceback (most recent call last)
# <ipython-input-46-1c87a983c286> in <module>
#      33 for i in tqdm.tnrange(len(public_salon[9661:])):
#      34     try:
# ---> 35         crawl_naver_map_v4(public_salon[9661:].iloc[i])
#      36 
#      37     except (exceptions.StaleElementReferenceException, exceptions.NoSuchElementException):

# <ipython-input-44-1b7f961ac223> in crawl_naver_map_v4(series)
#      20     fix_display_search = driver.find_element_by_id('searchCurr')
#      21     if fix_display_search.is_selected():
# ---> 22         fix_display_search.click()
#      23 
#      24     time.sleep(np.random.rand())

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py in click(self)
#      78     def click(self):
#      79         """Clicks the element."""
# ---> 80         self._execute(Command.CLICK_ELEMENT)
#      81 
#      82     def submit(self):

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py in _execute(self, command, params)
#     631             params = {}
#     632         params['id'] = self._id
# --> 633         return self._parent.execute(command, params)
#     634 
#     635     def find_element(self, by=By.ID, value=None):

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py in execute(self, driver_command, params)
#     319         response = self.command_executor.execute(driver_command, params)
#     320         if response:
# --> 321             self.error_handler.check_response(response)
#     322             response['value'] = self._unwrap_value(
#     323                 response.get('value', None))

# ~\AppData\Local\Continuum\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py in check_response(self, response)
#     240                 alert_text = value['alert'].get('text')
#     241             raise exception_class(message, screen, stacktrace, alert_text)
# --> 242         raise exception_class(message, screen, stacktrace)
#     243 
#     244     def _value_or_default(self, obj, key, default):

# ElementClickInterceptedException: Message: element click intercepted: Element <input type="checkbox" id="searchCurr" class="input_chk_curr nclicks(STA.offmap) btn_curr_on"> is not clickable at point (421, 23). Other element would receive the click: <div id="simplemodal-overlay" class="simplemodal-overlay" style="background-color: rgb(0, 0, 0); opacity: 0.6; height: 674px; width: 1036px; position: fixed; left: 0px; top: 0px; z-index: 40002;"></div>
#   (Session info: chrome=81.0.4044.113)

#==================================================================================
#==================================================================================

# start : 12709
# try row-num : 15913
# 검색어 : 우주헤어
# 검색결과 1건
# 검색 및 저장 성공 !!
# (성공 : 2404, 실패 : 0)

In [39]:
naver_search_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5376 entries, 0 to 5375
Data columns (total 8 columns):
업소ID       5376 non-null object
업소명        5376 non-null object
소재지전화번호    4394 non-null object
소재지도로명     5376 non-null object
카테고리       5376 non-null object
검색인덱스      5376 non-null int64
검색어        5376 non-null object
검색건수       5376 non-null int64
dtypes: int64(2), object(6)
memory usage: 336.1+ KB


In [40]:
naver_search_rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 6 columns):
지역        577 non-null object
업소명       577 non-null object
소재지도로명    577 non-null object
검색어       577 non-null object
검색건수      577 non-null int64
원인        577 non-null object
dtypes: int64(1), object(5)
memory usage: 27.2+ KB


In [303]:
pd.read_csv('./naver_map_seoul_salon_output_data_2020-04-09_2020-04-20.csv', index_col=0)

,업소ID,업소명,소재지전화번호,소재지도로명,카테고리,검색주소타입,검색인덱스,검색어,검색건수
0,s18570093,조희미용실,02-555-0157,서울특별시 강남구 남부순환로 2917 1층 111-가,"생활,편의 > 미용실",1,3,조희미용실 미용실,1
1,s10983326,정정원 헤어 룩,02-545-3809,서울특별시 강남구 선릉로116길 24,"생활,편의 > 미용실",1,4,정정원헤어룩 미용실,1
2,s11724823,박승철헤어스투디오 청담점,02-514-6167,서울특별시 강남구 삼성로 740,"생활,편의 > 미용실",1,5,박승철헤어스투디오 청담점 미용실,1
3,s11724823,박승철헤어스투디오 청담점,02-514-6167,서울특별시 강남구 삼성로 740,"생활,편의 > 미용실",1,6,박승철헤어스튜디오 미용실,1
4,s11693205,이가자헤어비스 청담본점,02-518-0077,서울특별시 강남구 압구정로 445 다이애나빌딩 3층,"생활,편의 > 미용실",1,7,이가자헤어비스 미용실,1
5,s13332566,준오헤어 압구정로데오 1호점,02-518-6050,서울특별시 강남구 선릉로 843,"생활,편의 > 미용실",1,8,준오헤어 미용실,2
6,s31399606,준오헤어 압구정로데오 2호점,02-516-2605,서울특별시 강남구 선릉로161길 16,"생활,편의 > 미용실",1,8,준오헤어 미용실,2
7,s18510764,헤어코코,02-541-1618,서울특별시 강남구 강남대로118길 33,"생활,편의 > 미용실",1,9,헤어코코 미용실,1
8,s11631944,이민헤어혼 본점,02-553-8755,서울특별시 강남구 도곡로 420,미용 > 미용실,1,12,이민헤어혼 미용실,1
9,s11631944,이민헤어혼 본점,02-553-8755,서울특별시 강남구 도곡로 420,미용 > 미용실,1,13,이민헤어혼 미용실,1
